# Batch models

Here we will describe how to perform batch modelling. This is a specific application where one needs many instances of the same kind of model, and those models do not take up much space. An example to keep in mind is a survey for which many of the objects are PSF limited galaxies, or faint stars; in this case a small cutout is sufficent and the same model (e.g. sersic galaxy model) may be applied to all of them. The reason one may want to do this is purely for performance improvements, because each operation is now being applied to an array instead of a single value, the code spends more time computing in the efficient numerical libraries instead of the slow python language.

Conceptually, a batch model is the same as a regular model, except in a few key places one replaces a single value with an array of values. The PyTorch backend is able to perform operations (addition, multiplication, etc.) on these arrays instead of the single values and effectively work with many models simultaneously. However, the restriction is that all of these batch models are sharing the same configuration. PSF convolution, for example, must be applied to all models or none. Similarly, if you wish to fix the inclination, you must do so for all the batch models.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import autoprof as ap
import numpy as np
import torch
from astropy.io import fits
import matplotlib.pyplot as plt
from time import time
%matplotlib inline

In [ ]:
hdu = fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=36.4684&dec=-45.889&size=100&layer=ls-dr9&pixscale=0.262&bands=r")
target_data = np.array(hdu[0].data, dtype = np.float64)

# Create a target object with specified pixelscale and zeropoint
target = ap.image.Target_Image(
    data = target_data,
    pixelscale = 0.262, # Every target image needs to know it's pixelscale in arcsec/pixel
    zeropoint = 22.5, # optionally, you can give a zeropoint to tell AutoProf what the pixel flux units are
    variance = np.ones(target_data.shape)/1e3, # set the variance for this image (in general it should be more accurate than this)
)

# The default AutoProf target plotting method uses log scaling in bright areas and histogram scaling in faint areas
fig3, ax3 = plt.subplots(figsize = (8,8))
ap.plots.target_image(fig3, ax3, target)
plt.show()

In [ ]:
model1 = ap.models.AutoProf_Model(
    name = "model1", 
    model_type = "sersic galaxy model", 
    parameters = {
        "center": [[7.,22.],[18.,4.],[18.,16.]], 
        "q": [0.8,0.8,0.8], 
        "PA": [0.,0.,0.], 
        "n": [2.,3.,4.], 
        "Re": [1.,1.,1.], 
        "Ie": [-1.,-1.,-1.], # notice that every parameter value is now an array of three values
    },
    window = ap.image.Window(shape = [0.262*20, 0.262*20], 
                             origin = torch.tensor(0.262*np.array([
                                 [int(7./0.262)-10., int(22./0.262)-10.],
                                 [int(18./0.262)-10.,int(4./0.262)-10],
                                 [int(18./0.262)-10,int(16./0.262)-10] # Notice that the origin changes for each batch model, but the shape CANNOT change, it must be the same for all of them
                             ]))),
    target = target, 
    batched = True, # This flag lets AutoProf know that this model is batched so that it can handle this properly
    integrate_mode = "full",
)

sample = model1()
print(sample)

fig4, ax4 = plt.subplots(1, 2, figsize = (16,7))
ap.plots.model_image(fig4, ax4[0], model1)
ap.plots.residual_image(fig4, ax4[1], model1)
plt.show()